In [ ]:
%autosave 0
import sys
sys.dont_write_bytecode = True
import os
home_dir = '/home/tim/Dropbox/' # home directory of repo downloaded from github
sys.path.append(home_dir)
from global_methods import *
from analytics_methods import *
from shortest_path.shortest_methods import *
from data.data_methods import *

# set parameters

In [ ]:
# set parent path to directory where data is stored (with observations as a subdirectory in this parent dir)
data_dir = '/home/tim/Dropbox/data/'


# **** the parameters below are static values for your reference, select from these to use in below cells

# will display data from these sensors in subplots later, in order from top-to-bottom left-to-right
# bounding boxes will be drawn on the previous sensor in list
sensor_names_all = [
    'SceneV1', # forward RGB scene (sunny, no weather) 144x256 resolution
    'BoxesV1', # ground truth bounding boxes of all objects in scene, will be drawn onto sensor above this
    'SegmentationV1', # ground truth color forward segementation 144x256 resolution
    'MasksV1', # ground truth dictionary of boolean array masks for each object in scene
    'DepthV1', # ground truth forward depth 144x256 resolution
    'DepthV2', # ground truth forward depth 36x64 resolution 
    'SceneV3', # forward RGB rain 144x256 resolution
    'SceneV4', # forward RGB snow 144x256 resolution
    'SceneV5', # forward RGB fog 144x256 resolution
]

# id_names are a unique identifier to tag specific subsets or versions of a data_dict
id_names_all = [
    'alpha', # most current stable data
    'beta', # experimental data -- will update to alpha when finished
    'demo', # quickly collected data to show
]

# these are valid ranges of coordinates that you can use
    # train is top half, test is bottom half, all is both halves
x_vals_train_blocks = [x for x in range(-60, 101, 2)]
x_vals_test_blocks = [x for x in range(-120, 1, 2)]
x_vals_all_blocks = [x for x in range(-120, 101, 2)]
y_vals_all_blocks = [y for y in range(-140, 141, 2)]
x_vals_train_airsimnh = [x for x in range(-10, 241, 2)]
x_vals_test_airsimnh = [x for x in range(-240, 11, 2)]
x_vals_all_airsimnh = [x for x in range(-240, 241, 2)]
y_vals_all_airsimnh = [y for y in range(-240, 241, 2)]
z_vals_2d, z_vals_3d = [-4], [-4] #[-16, -12, -8, -4] # working on z-vals outside of -4
yaw_vals_all = [0, 1, 2, 3]
#t_vals_all = [0] # working on temporal values to add to dictionary 

# rename titles on subplots in animation
sensor_psuedonames = {
    'DepthV1':'Depth', # forward depth 144x256 resolution
    'DepthV2':'Depth Downsampled', # forward depth 144x256 resolution
    'SceneV1':'RGB', # forward RGB 144x256 resolution
    'SceneV3':'Rain', # forward RGB rain 144x256 resolution
    'SceneV4':'Snow', # forward RGB snow 144x256 resolution
    'SceneV5':'Fog', # forward RGB fog 144x256 resolution
    'SegmentationV1':'Segmentation', # forward RGB fog 144x256 resolution
    'MasksV1':'Object Mask', # splits each individual object into bool array
}

# fetch and view data at a specific set of x, y, z, yaw coordinates

In [ ]:
# set coordinate ranges
x_vals = [x for x in range(-32, 32, 2)]
y_vals = [y for y in range(0, 2, 2)]
z_vals = [z for z in range(-4, 0, 4)]
yaw_vals = [0]
airsim_map = 'AirSimNH'
sensor_names = [
    'SceneV1',
    # 'BoxesV1',
    # 'SceneV3',
    # 'SceneV4',
    # 'SceneV5',
    'DepthV1',
    #'DepthV2',
    # 'SegmentationV1',
    # 'MasksV1',
]
id_names = ['alpha'] # only use stable data
resolution=[144,256] # height, width of observation arrays (images)

# fetch data from coordinates
location_data, location_animation = data_at_coordinates(airsim_map, sensor_names, id_names, x_vals, y_vals, z_vals, yaw_vals, 
                    make_animation=True, return_data=True, ncols=3, additional_subplots=0, resolution=resolution, sensor_psuedonames=sensor_psuedonames)

# display results
display_data_results(location_data, location_animation)

# read paths

In [ ]:
paths_path = f'{data_dir}astar_paths/v1/AirSimNH_2d_test.p'
paths_info, n_total_paths = load_paths(paths_path)

# view data along a predetermined random Astar path

In [ ]:
# set path to view (paths are ordered in increasing difficulty / length
path = paths_info['paths'][n_total_paths-16] # -16 is just a default arbitrary path I like
airsim_map = 'AirSimNH'
sensor_names = [
    'SceneV1',
    'BoxesV1',
    'SceneV3',
    'SceneV4',
    'SceneV5',
    'DepthV1',
    #'DepthV2',
    'SegmentationV1',
    'MasksV1',
]
id_names = ['alpha'] # only use stable data
resolution=[144,256] # height, width of observation arrays (images)

path_data, path_animation = data_at_path(data_dir, airsim_map, sensor_names, id_names, path, 
                 make_animation=True, return_data=True, ncols = 3, resolution=resolution, 
                 sensor_psuedonames=sensor_psuedonames, include_nulls=False)

# display results
display_data_results(path_data, path_animation)

# determine path between desired start/target point and view data

In [ ]:
motion = '2d' # 2d for horizontal plane only, 3d to allow drone to move up and down
start = [0, -80, -4] # x, y, z
target = [-80, 0, -4] # x, y, z
path = get_astar_path(motion, airsim_map, start, target)

# view data along path
path_data2, path_animation2 = data_at_path(data_dir, airsim_map, sensor_names, id_names, path, 
                 make_animation=True, return_data=True, ncols = 3, resolution=resolution, 
                 sensor_psuedonames=sensor_psuedonames, include_nulls=False)

# display results
display_data_results(path_data2, path_animation2)

# visualize test path(s) taken from a trained model

In [ ]:
airsim_map = 'Blocks'
motion = '2d'
model_dir = f'{home_dir}models/navigation_airsim_blocks_dqn_2d/'
test_dir = f'{model_dir}test_final/'
configuration_path = f'{model_dir}configuration.json'
configuration = json.load(open(configuration_path, 'r'))
actions = configuration['components']['Actor']['actions_components'] # list of actions DQN can take
episodes = read_evaluations(f'{test_dir}states/')[0]
sensor_names = ['DepthV2'] # sensor data to view at each step
resolution = [36, 64] # highest resolution of image data to display [height, width]
id_names = ['alpha'] # id of which data to use (alpha is stable build)

# view data at given episode
episode_idx = 777
episode = episodes[episode_idx]
episode_data, episode_animation = data_at_episode(data_dir, airsim_map, sensor_names, id_names, 
                                      episode, actions, make_animation=True, return_data=True,
                                                 resolution=resolution, 
                                                  sensor_psuedonames=sensor_psuedonames)

# display results
display_data_results(episode_data, episode_animation)

episode_data